In [32]:
import os
import json
import pandas as pd

def collect_data(main_folder):
    """ Collects the minimum test loss and corresponding parameters across all subfolders in the main folder. """
    results = []
    context_size = 0
    # Traverse through each subfolder in the main folder
    for subfolder in os.listdir(main_folder):
        subfolder_path = os.path.join(main_folder, subfolder)
        progress_file = os.path.join(subfolder_path, 'progress.csv')
        params_file = os.path.join(subfolder_path, 'params.json')
        
        # Check if both necessary files exist
        if os.path.exists(progress_file) and os.path.exists(params_file):
            try:
                # Read progress.csv and find the minimum test loss
                data = pd.read_csv(progress_file)
                data.fillna(0, inplace=True)
                # Read params.json
                with open(params_file, 'r') as file:
                    params = json.load(file)
                    # Collect required params and the corresponding test loss
                    result = {}
                    if 'master_neurons' in params:
                        result['master_neurons'] = params.get('master_neurons')
                    if 'context_store' in params:
                        result['store_embedding_neurons'] = params.get('context_store')
                    if 'context' in params:
                        result['context_neurons'] = params.get('context')
                    result['learning_rate'] = params.get('learning_rate')

                    result['best_train_loss'] = data['train_loss'].min()
                    result['dev_loss(at best_train)'] = data[data['train_loss'] == result['best_train_loss']]['dev_loss'].iloc[0]
                    result['best_dev_loss'] = data['dev_loss'].min()
                    result['train_loss(at best_dev)'] = data[data['dev_loss'] == result['best_dev_loss']]['train_loss'].iloc[0]
                    results.append(result)
            except Exception as e:
                print(f"Error processing files in {subfolder_path}: {e}")

    return results

def create_results_table(main_folder, remove_rup_hyperparams = False):
    """ Creates a table of the minimum test losses for each combination of learning_rate, context_size, and samples. """
    data = collect_data(main_folder)
    if data:
        # Create DataFrame from collected data
        df = pd.DataFrame(data)
        # Group by the parameters and find the row with the minimum dev_loss
        columns_to_sort = ['master_neurons', 'store_embedding_neurons', 'context_neurons', 'learning_rate']
        existing_columns = [col for col in columns_to_sort if col in df.columns]
        sort_order = [True] * len(existing_columns)
        sorted_result_df = df.sort_values(by=existing_columns, ascending=sort_order)
        
        if remove_rup_hyperparams == True:
            sorted_result_df = sorted_result_df.drop_duplicates(subset=existing_columns, keep='first')
        
        pd.set_option('display.float_format', lambda x: '%.5f' % x)
        return sorted_result_df
        # print(sorted_result_df.to_string(index=False))
    else:
        print("No data collected. Check the contents of your directories.")


In [33]:
df = create_results_table('/user/ml4723/Prj/NIC/ray_results_real_symmetry_GNN_real_data/run_2024-07-18_05-16-26', True)
df = df[(df['best_train_loss'] <= -250) | (df['best_dev_loss'] <= -250)]
print(df.to_string(index=False))

 store_embedding_neurons  context_neurons  learning_rate  best_train_loss  dev_loss(at best_train)  best_dev_loss  train_loss(at best_dev)
                      16               16        0.00010       -371.07332               -318.13960     -320.26870               -370.46277
                      16               16        0.00100       -363.71438               -303.73568     -322.61008               -361.45815
                      16               16        0.00100       -371.31272               -324.28394     -325.36754               -368.90999
                      16               32        0.00010       -380.89409               -332.31020     -333.39084               -380.47350
                      16               64        0.00100       -388.41780               -340.98204     -343.72932               -385.36803
                      16              128        0.01000       -309.59894               -286.68678     -286.68678               -309.59894
                      16   

In [13]:
print(create_results_table('/user/ml4723/Prj/NIC/ray_results_real_symmetry_aware_real_data/run_2024-07-16_21-40-19').to_string(index=False))

 learning_rate  best_train_loss  dev_loss(at best_train)  best_dev_loss  train_loss(at best_dev)
       0.00001         -2.97386                 -2.98470       -2.98470                 -2.97386
       0.00001       -109.47387                 -3.51074      -10.44436                -10.55865
       0.00001       -285.53259               -177.24217     -177.93759               -267.27114
       0.00001         -2.10252                 -2.11105       -2.11105                 -2.10252
       0.00010       -294.44796               -196.24351     -196.61141               -293.95203
       0.00010       -384.92125               -316.62712     -322.38249               -380.26535
       0.00010       -387.20982               -316.48830     -322.87196               -371.44722
       0.00010       -328.08413               -258.87899     -258.87899               -328.08413
       0.00100          0.00000                  0.00000        0.00000                  0.00000
       0.00100       -364.8415